# Competitive race

Before you start working with this notebook, remember to:

* Ensure that `imagemagick` is installed (e.g., `conda install imagemagick`) for the command-line utility `convert`.
* Ensure that the directory `DP04-Race-Submissions` has student submissions.
* Ensure that the file `students.json` (an array of dictionaries with keys `netid`, `first_name`, `last_name`, and `dp4_partner`) has the student roster.

You may also need to activate a conda environment with `python-control`, depending on whether or not students expect that module to be available.

Notes:

* If text stops being printed to the notebook, do `Kernel -> Reconnect`.

Import modules and configure the notebook.

In [ ]:
import os
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import secrets
import json
import shutil
import subprocess
import ae353_drone
import importlib
importlib.reload(ae353_drone)

Prevent students from importing `ae353_drone` in their own code.

In [ ]:
import sys
sys.modules['ae353_drone'] = None

Create and print seed so it is possible to reproduce the results.

In [ ]:
seed = secrets.randbits(32)
print(seed)

Create simulator.

In [ ]:
simulator = ae353_drone.Simulator(display=True, seed=seed)

Copy student submissions and student roster.

In [ ]:
# Get string with current date and time
datetimestr = datetime.now().strftime('%Y%m%dT%H%M%S')

# Copy student submissions
srcdir_designs = f'{datetimestr}-designs'
results = shutil.copytree(
    'DP04-Race-Submissions',
    srcdir_designs,
)

# Copy student roster
filename_students = f'{datetimestr}-students.json'
results = shutil.copyfile(
    'students.json',
    filename_students,
)

Load student roster.

In [ ]:
with open(filename_students, 'r') as infile:
    students = json.load(infile)

def get_student(students, netid):
    for student in students:
        if student['netid'] == netid:
            return student
    return None

def get_partners(students, student):
    partner_netids = np.array(student['dp4_partner']).flatten().tolist()
    partner_students = []
    for netid in partner_netids:
        partner_students.append(get_student(students, netid))
    return partner_students

The amount of time for which to run each simulation.

In [ ]:
max_time = 45.

Make sure all files in source directory have lower-case names.

In [ ]:
srcdir = srcdir_designs
for file in os.listdir(srcdir):
    os.rename(os.path.join(srcdir, file), os.path.join(srcdir, file.lower()))

Make sure all PNG files in source directory really are PNG files.

In [ ]:
srcdir = srcdir_designs
template_image = 'question_mark.png'
for file in os.listdir(srcdir):
    if file.endswith('.png'):
        completed_process = subprocess.run([
                    'convert',
                    os.path.join(srcdir, file),
                    os.path.join(srcdir, file),
                ], capture_output=True)
        if completed_process.returncode != 0:
            print(f'   ** FAILED on {file} (returncode: {completed_process.returncode}), replacing with template')
            shutil.copyfile(template_image, os.path.join(srcdir, file))

Look for and move submissions with names that do not have the form `netid.py`.

In [ ]:
srcdir = srcdir_designs
for file in os.listdir(srcdir):
    if file.endswith('.py'):
        netid = file.removesuffix('.py')
        student = get_student(students, netid)
        if student is None:
            print(f'  ** BAD CODE NAME - {file} moved to "bad-code-name-{file}"')
            src = os.path.join(srcdir, file)
            dst = os.path.join(srcdir, f'bad-code-name-{file}')
            shutil.move(src, dst)

Look for and move duplicate submissions.

In [ ]:
netids_to_email = []
teams = []
srcdir = srcdir_designs
for file in os.listdir(srcdir):
    if file.endswith('.py'):
        netid = file.removesuffix('.py')
        student = get_student(students, netid)
        if student is None:
            continue
        team = student['dp4_group_name']
        if team in teams:
            name = f'{student["first_name"]} {student["last_name"]}'
            print(f'  ** DUPLICATE SUBMISSION by {name} for {team}\n       (moved to "duplicate-{file}")')
            netids_to_email.append(student['netid'] + '@illinois.edu')
            partners = get_partners(students, student)
            for partner in partners:
                netids_to_email.append(partner['netid'] + '@illinois.edu')
            src = os.path.join(srcdir, file)
            dst = os.path.join(srcdir, f'duplicate-{file}')
            shutil.move(src, dst)
        teams.append(team)

if len(netids_to_email) > 0:
    print(f'\nSTUDENTS TO EMAIL ({len(netids_to_email)}):\n')
    print(' ' + ', '.join(netids_to_email))

Load drones from source directory, overriding the maximum allowable number.

In [ ]:
simulator.clear_drones()
failures = simulator.load_drones(srcdir_designs, no_max_num_drones=True)

List disqualified drones.

In [ ]:
netids_to_email = []
print(f'DISQUALIFIED ({len(failures)}):\n')
for failure in failures:
    if failure.startswith('bad-code-name'):
        continue
    
    if failure.startswith('duplicate'):
        continue
    
    student = get_student(students, failure)
    if student is None:
        name = ''
    else:
        student['dp4_status'] = 'disqualified'
        name = f'{student["first_name"]} {student["last_name"]}'
        netids_to_email.append(student['netid'] + '@illinois.edu')
        partners = get_partners(students, student)
        for partner in partners:
            partner['dp4_status'] = 'disqualified'
            name += f' and {partner["first_name"]} {partner["last_name"]}'
            netids_to_email.append(partner['netid'] + '@illinois.edu')
    print(f' {failure:20s} : {name}')

if len(netids_to_email) > 0:
    print(f'\nSTUDENTS TO EMAIL ({len(netids_to_email)}):\n')
    print(' ' + ', '.join(netids_to_email))

List qualified drones.

In [ ]:
print(f'QUALIFIED ({len(simulator.drones)}):\n')
for drone in simulator.drones:
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    student['dp4_status'] = 'qualified'
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        partner['dp4_status'] = 'qualified'
        name += f' and {partner["first_name"]} {partner["last_name"]}'
    print(f' {drone["name"]:15s} : {name}')

List non-submissions.

In [ ]:
netids_to_email = []

print(f'NON-SUBMISSIONS:\n')
for student in students:
    if not 'dp4_status' in student:
        student['dp4_status'] = 'did not submit'
        name = f'{student["first_name"]} {student["last_name"]}'
        netids_to_email.append(student['netid'] + '@illinois.edu')
        partners = get_partners(students, student)
        for partner in partners:
            partner['dp4_status'] = 'did not submit'
            name += f' and {partner["first_name"]} {partner["last_name"]}'
            netids_to_email.append(partner['netid'] + '@illinois.edu')
        print(f' {name}')

if len(netids_to_email) > 0:
    print(f'\nSTUDENTS TO EMAIL ({len(netids_to_email)}):\n')
    print(' ' + ', '.join(netids_to_email))

Save results of qualification to file.

In [ ]:
with open(f'{datetimestr}-students-qualification.json', 'w') as outfile:
    json.dump(students, outfile, indent=4)

Define functions to show results.

In [ ]:
benign_failures = [
    'Inactive.',
    'Out of bounds.',
]

def get_netids_to_email(drone_name, students):
    student = get_student(students, drone_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone_name}')
    
    netids_to_email = []
    netids_to_email.append(student['netid'] + '@illinois.edu')
    partners = get_partners(students, student)
    for partner in partners:
        netids_to_email.append(partner['netid'] + '@illinois.edu')
    
    return netids_to_email

def get_student_name(drone_name, students):
    student = get_student(students, drone_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone_name}')
    
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f' and {partner["first_name"]} {partner["last_name"]}'
        
    return name

def disqualify_student(drone_name, drone_error, students):
    student = get_student(students, drone_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone_name}')
    student['dp4_status'] = 'disqualified'
    student['dp4_error'] = drone_error
    partners = get_partners(students, student)
    for partner in partners:
        partner['dp4_status'] = 'disqualified'
        partner['dp4_error'] = drone_error

def get_results(simulator, students):
    netids_to_email = []
    finished = []
    still_running = []
    failed = []
    errors = ''
    results = ''
    for drone in simulator.drones:
        if drone['finish_time'] is not None:
            finished.append((drone, drone['finish_time']))
        elif drone['running']:
            still_running.append(drone)
        else:
            failed.append(drone)
            errors += f'======================\n{drone["error"]}\n======================\n\n'
    finished = sorted(finished, key=lambda f: f[1])
    
    results += 'FINISHED\n'
    for d in finished:
        drone = d[0]
        drone_name = drone['name']
        student_name = get_student_name(drone_name, students)
        results += f' {d[1]:6.2f} : {drone_name:20s} : {student_name}\n'

    results += '\nSTILL RUNNING\n'
    for d in still_running:
        drone = d
        drone_name = drone['name']
        student_name = get_student_name(drone_name, students)
        results += f'        : {drone_name:20s} : {student_name}\n'
    
    results += '\nINACTIVE OR OUT OF BOUNDS\n'
    for d in failed:
        drone = d
        drone_name = drone['name']
        if drone['error'] in benign_failures:
            student_name = get_student_name(drone_name, students)
            results += f'        : {drone_name:20s} : {student_name}\n'
    
    results += '\nFAILED\n'
    for d in failed:
        drone = d
        drone_name = drone['name']
        if drone['error'] not in benign_failures:
            disqualify_student(drone_name, drone['error'], students)
            student_name = get_student_name(drone_name, students)
            netids_to_email.extend(get_netids_to_email(drone_name, students))
            results += f'        : {drone_name:20s} : {student_name}\n'
    
    results += '\nERRORS (REASONS FOR FAILURE)\n\n'
    results += errors
    
    results += '\nNETIDS TO EMAIL ABOUT FAILURE\n\n'
    results += (' ' + ', '.join(netids_to_email))
    
    return results

Choose number of drones to race in each semifinal.

In [ ]:
num_drones_per_semifinal = int(np.ceil(np.sqrt(len(simulator.drones))))
num_semifinals = int(np.ceil(len(simulator.drones) / num_drones_per_semifinal))
print(f'There will be at most {num_drones_per_semifinal} drones in each of {num_semifinals} semifinals.')

Create semifinal races.

In [ ]:
# Get list of qualified racers
qualified = [drone['name'] for drone in simulator.drones]

# Copy list of qualified racers for later use
list_of_qualified_racers = qualified.copy()

# Shuffle order of this list
simulator.rng.shuffle(qualified)

# Create each race
num_races = 0
while True:
    racers = qualified[-num_drones_per_semifinal:]
    qualified = qualified[:-num_drones_per_semifinal]
    
    srcdir = srcdir_designs
    dstdir = f'{datetimestr}-comp-semifinal-{num_races}'
    os.mkdir(dstdir)
    for racer in racers:
        shutil.copyfile(os.path.join(srcdir, f'{racer}.py'), os.path.join(dstdir, f'{racer}.py'))
        shutil.copyfile(os.path.join(srcdir, f'{racer}.png'), os.path.join(dstdir, f'{racer}.png'))
    
    num_races += 1
    if len(qualified) == 0:
        break

# Say how many semifinal races were created
print(f'Created {num_races} semifinal races')

# Create directory for final race
os.mkdir(f'{datetimestr}-comp-final')

Initialize the race index.

In [ ]:
index_of_race = 0

## Semifinal races

This section of the notebook should be evaluated once for each semifinal race.

Print index of current race.

In [ ]:
print(f'Running semifinal race {index_of_race + 1} / {num_races}')

Ready...

In [ ]:
# Name of directory with racers
srcdir = f'{datetimestr}-comp-semifinal-{index_of_race}'

# Clear drones
simulator.clear_drones()

# Move rings
simulator.place_rings()

# Load drones
simulator.load_drones(srcdir)

# Reset
simulator.reset()

Steady...

In [ ]:
simulator.camera_contestview()

num_drones = len(simulator.drones)
num_columns = 3
num_rows = np.ceil(num_drones / num_columns).astype(int)
fig, axs = plt.subplots(num_rows, num_columns, figsize=(12, 4 * num_rows))
[ax.set_axis_off() for ax in axs.flatten()]
for ax, drone in zip(axs.flatten(), simulator.drones):
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f'\n{partner["first_name"]} {partner["last_name"]}'
    im = plt.imread(os.path.join(srcdir, f'{drone["name"]}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'{drone["name"]}\n{name}', fontsize=14)
    ax.axis('equal')

fig.tight_layout(h_pad=5)

Go!

In [ ]:
start_time = time.time()
simulator.run(max_time=max_time, print_debug=True)
print(f'real time elapsed: {time.time() - start_time}')

Find winner.

In [ ]:
winning_name = None
winning_time = np.inf
for drone in simulator.drones:
    if drone['finish_time'] is None:
        continue
    if drone['finish_time'] < winning_time:
        winning_name = drone['name']
        winning_time = drone['finish_time']

if winning_name is None:
    print(f'There was no winner (nobody finished).')
else:
    print(f'The winner was {winning_name} with time {winning_time:.2f} seconds')
    srcdir = f'{datetimestr}-comp-semifinal-{index_of_race}'
    dstdir = f'{datetimestr}-comp-final'
    shutil.copyfile(os.path.join(srcdir, f'{winning_name}.py'), os.path.join(dstdir, f'{winning_name}.py'))
    shutil.copyfile(os.path.join(srcdir, f'{winning_name}.png'), os.path.join(dstdir, f'{winning_name}.png'))
    student = get_student(students, winning_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {winning_name}')
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f'\n{partner["first_name"]} {partner["last_name"]}'
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_axis_off()
    im = plt.imread(os.path.join(srcdir, f'{winning_name}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'WINNER ({winning_time:.2f} seconds)\n\n{winning_name}\n{name}', fontsize=24)
    ax.axis('equal')

Show all results.

In [ ]:
results = get_results(simulator, students)
print(results)

with open(f'{datetimestr}-comp-semifinal-{index_of_race}.txt', 'w') as f:
    f.write(results)

Increment index of race.

In [ ]:
index_of_race += 1
if index_of_race == num_races:
    print('STOP! YOU ARE DONE WITH THE SEMIFINALS')

## Final race

Ready...

In [ ]:
# Name of directory with racers
srcdir = f'{datetimestr}-comp-final'

# Clear drones
simulator.clear_drones()

# Move rings
simulator.place_rings()

# Load drones
simulator.load_drones(srcdir)

# Reset
simulator.reset()

Steady...

In [ ]:
simulator.camera_contestview()

num_drones = len(simulator.drones)
num_columns = 3
num_rows = np.ceil(num_drones / num_columns).astype(int)
fig, axs = plt.subplots(num_rows, num_columns, figsize=(12, 4 * num_rows))
[ax.set_axis_off() for ax in axs.flatten()]
for ax, drone in zip(axs.flatten(), simulator.drones):
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f'\n{partner["first_name"]} {partner["last_name"]}'
    im = plt.imread(os.path.join(srcdir, f'{drone["name"]}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'{drone["name"]}\n{name}', fontsize=14)
    ax.axis('equal')

fig.tight_layout(h_pad=5)

Go!

In [ ]:
start_time = time.time()
simulator.run(max_time=max_time, print_debug=True)
print(f'real time elapsed: {time.time() - start_time}')

Find winner.

In [ ]:
winning_name = None
winning_time = np.inf
for drone in simulator.drones:
    if drone['finish_time'] is None:
        continue
    if drone['finish_time'] < winning_time:
        winning_name = drone['name']
        winning_time = drone['finish_time']

if winning_name is None:
    print(f'There was no winner (nobody finished).')
else:
    print(f'The winner was {winning_name} with time {winning_time:.2f} seconds')
    student = get_student(students, winning_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {winning_name}')
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f'\n{partner["first_name"]} {partner["last_name"]}'
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_axis_off()
    im = plt.imread(os.path.join(srcdir, f'{winning_name}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'WINNER ({winning_time:.2f} seconds)\n\n{winning_name}\n{name}', fontsize=24)
    ax.axis('equal')

Show all results.

In [ ]:
results = get_results(simulator, students)
print(results)

with open(f'{datetimestr}-comp-final.txt', 'w') as f:
    f.write(results)

## Free-for-all (just for fun)

Define index of free-for-all race (do this only once).

In [ ]:
index_of_race = 0

Create free-for-all race.

In [ ]:
# Get list of racers
racers = list_of_qualified_racers.copy()

# Shuffle the order of this list
simulator.rng.shuffle(racers)

# Keep only as many racers as one simulation can handle
racers = racers[:simulator.max_num_drones]

# Create directory with racers
srcdir = srcdir_designs
dstdir = f'{datetimestr}-free-for-all-{index_of_race}'
os.mkdir(dstdir)
for racer in racers:
    shutil.copyfile(os.path.join(srcdir, f'{racer}.py'), os.path.join(dstdir, f'{racer}.py'))
    shutil.copyfile(os.path.join(srcdir, f'{racer}.png'), os.path.join(dstdir, f'{racer}.png'))

Ready...

In [ ]:
# Name of directory with racers
srcdir = f'{datetimestr}-free-for-all-{index_of_race}'

# Clear drones
simulator.clear_drones()

# Move rings
simulator.place_rings()

# Load drones
simulator.load_drones(srcdir)

# Reset
while True:
    try:
        simulator.reset()
        break
    except Exception:
        print('Reset failed - trying again...')
        continue

Steady...

In [ ]:
simulator.camera_contestview()

num_drones = len(simulator.drones)
num_columns = 3
num_rows = np.ceil(num_drones / num_columns).astype(int)
fig, axs = plt.subplots(num_rows, num_columns, figsize=(12, 4 * num_rows))
[ax.set_axis_off() for ax in axs.flatten()]
for ax, drone in zip(axs.flatten(), simulator.drones):
    student = get_student(students, drone['name'])
    if student is None:
        raise Exception(f'could not find student for this drone name: {drone["name"]}')
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f'\n{partner["first_name"]} {partner["last_name"]}'
    im = plt.imread(os.path.join(srcdir, f'{drone["name"]}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'{drone["name"]}\n{name}', fontsize=14)
    ax.axis('equal')

fig.tight_layout(h_pad=5)

Go!

In [ ]:
start_time = time.time()
simulator.run(max_time=max_time, print_debug=True)
print(f'real time elapsed: {time.time() - start_time}')

Find winner.

In [ ]:
winning_name = None
winning_time = np.inf
for drone in simulator.drones:
    if drone['finish_time'] is None:
        continue
    if drone['finish_time'] < winning_time:
        winning_name = drone['name']
        winning_time = drone['finish_time']

if winning_name is None:
    print(f'There was no winner (nobody finished).')
else:
    print(f'The winner was {winning_name} with time {winning_time:.2f} seconds')
    student = get_student(students, winning_name)
    if student is None:
        raise Exception(f'could not find student for this drone name: {winning_name}')
    name = f'{student["first_name"]} {student["last_name"]}'
    partners = get_partners(students, student)
    for partner in partners:
        name += f'\n{partner["first_name"]} {partner["last_name"]}'
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_axis_off()
    im = plt.imread(os.path.join(srcdir, f'{winning_name}.png'))
    ax.imshow(im, aspect='equal')
    ax.set_title(f'FREE-FOR-ALL WINNER ({winning_time:.2f} seconds)\n\n{winning_name}\n{name}', fontsize=24)
    ax.axis('equal')

Show all results.

In [ ]:
results = get_results(simulator, students)
print(results)

with open(f'{datetimestr}-free-for-all-{index_of_race}.txt', 'w') as f:
    f.write(results)

Increment index of race.

In [ ]:
index_of_race += 1

## Save final status

In [ ]:
with open(f'{datetimestr}-students-qualification-final.json', 'w') as outfile:
    json.dump(students, outfile, indent=4)